In [1]:
import pandas as pd
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
from sklearn.impute import SimpleImputer
import os.path
# Harmonization
from sklearn.model_selection import train_test_split
from neurocombat_sklearn import CombatModel
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error

import pickle
from tools import load_data, model_fit

In [2]:
X, y, site = load_data()

2022-07-06 01:38:42.577488: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from sklearn.utils import shuffle, resample

In [4]:
from sklearn.metrics import r2_score, median_absolute_error

In [5]:
model_dict = {
  "cnn_lenet": {"model": cnn_lenet, "lr": 0.001}, 
  # "mlp4": {"model": mlp4, "lr": 0.001},
  # "cnn_vgg": {"model": cnn_vgg, "lr": 0.001},
  # "lstm1v0": {"model": lstm1v0, "lr": 0.01},
  # "lstm1": {"model": lstm1, "lr": 0.01},
  # "lstm2": {"model": lstm2, "lr": 0.01},
  # "blstm1": {"model": blstm1, "lr": 0.01},
  # "blstm2": {"model": blstm1, "lr": 0.01},
  # "lstm_fcn": {"model": lstm_fcn, "lr": 0.01},
#   "cnn_resnet": {"model": cnn_resnet, "lr": 0.01}
             }

In [6]:
n_runs = 10

In [7]:
# Generated once with this code and then hard-coded:
# seeds = np.array([np.abs(np.floor(np.random.randn()*1000)) for ii in range(n_runs)], dtype=int)

seeds = np.array([484, 645, 714, 244, 215, 1502, 1334, 1576, 469, 1795])

In [8]:
def fit_and_eval(model, X, y, random_state, batch_size=128, n_epochs=1000, augment=None, train_size=None):
    model_func = model_dict[model]["model"]
    lr = model_dict[model]["lr"]
    X_train, X_test, y_train, y_test, site_train, site_test = train_test_split(X, y, site, test_size=0.2, random_state=random_state)
    imputer = SimpleImputer(strategy="median")
    # If train_size is set, select train_size subjects to be the training data:
    if train_size is not None:
        X_train, y_train, site_train = resample(X_train, y_train, site_train, replace=False, n_samples=train_size, random_state=random_state)
    
    # Impute train and test separately:
    X_train = np.concatenate([imputer.fit_transform(X_train[..., ii])[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([imputer.fit_transform(X_test[..., ii])[:, :, None] for ii in range(X_test.shape[-1])], -1)
    # Combat
    X_train = np.concatenate([CombatModel().fit_transform(X_train[..., ii], site_train[:, None], None, None)[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([CombatModel().fit_transform(X_test[..., ii], site_test[:, None], None, None)[:, :, None] for ii in range(X_test.shape[-1])], -1)
    
    trained = model_fit(model_func, X_train, y_train, lr, 
                        batch_size=batch_size, n_epochs=n_epochs)
    metric = []
    value = []
    
    y_pred = trained.predict(X_test)
    metric.append("mae")
    value.append(mean_absolute_error(y_test, y_pred))
    metric.append("mad")
    value.append(median_absolute_error(y_test, y_pred))
    metric.append("r2")
    value.append(r2_score(y_test, y_pred))
    
    result = {'Model': [model] * len(metric),
              'Metric': metric,
              'Value': value}
    
    return pd.DataFrame(result), pd.DataFrame(dict(y_pred=y_pred.squeeze(), y_test=y_test))

In [9]:
dfs_eval = []
dfs_pred = []

In [10]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[ii],
            train_size=350)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("lenet_4_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("lenet_4_pred.csv")

##################################################
model:  cnn_lenet
run:  0
pooling layers: 4
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 conv1d (Conv1D)             (None, 100, 6)            1302      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 50, 6)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 50, 16)            304       
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 25, 16)           0         
 1D)                                                             
                                

Epoch 21/1000
3/3 [==============================] - 0s 23ms/step - loss: 18.8900 - mean_squared_error: 18.8900 - rmse: 4.3463 - mean_absolute_error: 3.4738 - val_loss: 13.5523 - val_mean_squared_error: 13.5523 - val_rmse: 3.6813 - val_mean_absolute_error: 2.9425 - lr: 0.0010
Epoch 22/1000
3/3 [==============================] - 0s 23ms/step - loss: 17.6074 - mean_squared_error: 17.6074 - rmse: 4.1961 - mean_absolute_error: 3.2541 - val_loss: 12.7384 - val_mean_squared_error: 12.7384 - val_rmse: 3.5691 - val_mean_absolute_error: 2.8973 - lr: 0.0010
Epoch 23/1000
3/3 [==============================] - 0s 24ms/step - loss: 18.7182 - mean_squared_error: 18.7182 - rmse: 4.3265 - mean_absolute_error: 3.3990 - val_loss: 12.6029 - val_mean_squared_error: 12.6029 - val_rmse: 3.5501 - val_mean_absolute_error: 2.8891 - lr: 0.0010
Epoch 24/1000
3/3 [==============================] - 0s 27ms/step - loss: 15.2520 - mean_squared_error: 15.2520 - rmse: 3.9054 - mean_absolute_error: 3.1844 - val_loss: 

Epoch 49/1000
3/3 [==============================] - 0s 26ms/step - loss: 13.4299 - mean_squared_error: 13.4299 - rmse: 3.6647 - mean_absolute_error: 2.8737 - val_loss: 12.2024 - val_mean_squared_error: 12.2024 - val_rmse: 3.4932 - val_mean_absolute_error: 2.8620 - lr: 2.5000e-04
Epoch 50/1000
3/3 [==============================] - 0s 25ms/step - loss: 15.6937 - mean_squared_error: 15.6937 - rmse: 3.9615 - mean_absolute_error: 3.0852 - val_loss: 12.2171 - val_mean_squared_error: 12.2171 - val_rmse: 3.4953 - val_mean_absolute_error: 2.8627 - lr: 2.5000e-04
Epoch 51/1000
3/3 [==============================] - 0s 25ms/step - loss: 17.1398 - mean_squared_error: 17.1398 - rmse: 4.1400 - mean_absolute_error: 3.3173 - val_loss: 12.2083 - val_mean_squared_error: 12.2083 - val_rmse: 3.4940 - val_mean_absolute_error: 2.8622 - lr: 2.5000e-04
Epoch 52/1000
3/3 [==============================] - 0s 23ms/step - loss: 15.3791 - mean_squared_error: 15.3791 - rmse: 3.9216 - mean_absolute_error: 3.1081 

3/3 [==============================] - 0s 28ms/step - loss: 15.8721 - mean_squared_error: 15.8721 - rmse: 3.9840 - mean_absolute_error: 3.1764 - val_loss: 11.8587 - val_mean_squared_error: 11.8587 - val_rmse: 3.4436 - val_mean_absolute_error: 2.8394 - lr: 1.2500e-04
Epoch 78/1000
3/3 [==============================] - 0s 28ms/step - loss: 14.9399 - mean_squared_error: 14.9399 - rmse: 3.8652 - mean_absolute_error: 3.0868 - val_loss: 12.0601 - val_mean_squared_error: 12.0601 - val_rmse: 3.4728 - val_mean_absolute_error: 2.8515 - lr: 1.2500e-04
Epoch 79/1000
3/3 [==============================] - 0s 27ms/step - loss: 18.1352 - mean_squared_error: 18.1352 - rmse: 4.2585 - mean_absolute_error: 3.4131 - val_loss: 12.4369 - val_mean_squared_error: 12.4369 - val_rmse: 3.5266 - val_mean_absolute_error: 2.8722 - lr: 1.2500e-04
Epoch 80/1000
3/3 [==============================] - 0s 24ms/step - loss: 15.1674 - mean_squared_error: 15.1674 - rmse: 3.8945 - mean_absolute_error: 3.0503 - val_loss: 12

1/3 [=========>....................] - ETA: 0s - loss: 20.8417 - mean_squared_error: 20.8417 - rmse: 4.5653 - mean_absolute_error: 3.6029
Epoch 00105: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
3/3 [==============================] - 0s 33ms/step - loss: 17.7801 - mean_squared_error: 17.7801 - rmse: 4.2166 - mean_absolute_error: 3.3041 - val_loss: 12.1151 - val_mean_squared_error: 12.1151 - val_rmse: 3.4807 - val_mean_absolute_error: 2.8533 - lr: 6.2500e-05
run:  1
pooling layers: 4
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 conv1d_4 (Conv1D)           (None, 100, 6)            1302      
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 50, 6)            0         
 1D)      

...

1/3 [=========>....................] - ETA: 0s - loss: 19.5181 - mean_squared_error: 19.5181 - rmse: 4.4179 - mean_absolute_error: 3.5760
Epoch 00249: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.
3/3 [==============================] - 0s 18ms/step - loss: 16.1300 - mean_squared_error: 16.1300 - rmse: 4.0162 - mean_absolute_error: 3.1430 - val_loss: 9.1822 - val_mean_squared_error: 9.1822 - val_rmse: 3.0302 - val_mean_absolute_error: 2.5215 - lr: 3.1250e-05
Epoch 250/1000
3/3 [==============================] - 0s 18ms/step - loss: 14.0399 - mean_squared_error: 14.0399 - rmse: 3.7470 - mean_absolute_error: 2.9684 - val_loss: 9.1885 - val_mean_squared_error: 9.1885 - val_rmse: 3.0313 - val_mean_absolute_error: 2.5225 - lr: 1.5625e-05
Epoch 251/1000
3/3 [==============================] - 0s 19ms/step - loss: 14.1107 - mean_squared_error: 14.1107 - rmse: 3.7564 - mean_absolute_error: 2.9685 - val_loss: 9.1943 - val_mean_squared_error: 9.1943 - val_rmse: 3.0322 - val_m

3/3 [==============================] - 0s 19ms/step - loss: 12.6484 - mean_squared_error: 12.6484 - rmse: 3.5565 - mean_absolute_error: 2.8267 - val_loss: 9.1521 - val_mean_squared_error: 9.1521 - val_rmse: 3.0253 - val_mean_absolute_error: 2.5162 - lr: 1.5625e-05
Epoch 278/1000
3/3 [==============================] - 0s 18ms/step - loss: 13.6577 - mean_squared_error: 13.6577 - rmse: 3.6956 - mean_absolute_error: 2.9206 - val_loss: 9.1516 - val_mean_squared_error: 9.1516 - val_rmse: 3.0252 - val_mean_absolute_error: 2.5160 - lr: 1.5625e-05
Epoch 279/1000
3/3 [==============================] - 0s 18ms/step - loss: 14.0025 - mean_squared_error: 14.0025 - rmse: 3.7420 - mean_absolute_error: 2.9180 - val_loss: 9.1521 - val_mean_squared_error: 9.1521 - val_rmse: 3.0252 - val_mean_absolute_error: 2.5157 - lr: 1.5625e-05
Epoch 280/1000
3/3 [==============================] - 0s 20ms/step - loss: 13.2946 - mean_squared_error: 13.2946 - rmse: 3.6462 - mean_absolute_error: 2.8664 - val_loss: 9.152

3/3 [==============================] - 0s 17ms/step - loss: 13.1595 - mean_squared_error: 13.1595 - rmse: 3.6276 - mean_absolute_error: 2.8589 - val_loss: 9.1981 - val_mean_squared_error: 9.1981 - val_rmse: 3.0328 - val_mean_absolute_error: 2.5237 - lr: 7.8125e-06
Epoch 306/1000
3/3 [==============================] - 0s 20ms/step - loss: 12.9702 - mean_squared_error: 12.9702 - rmse: 3.6014 - mean_absolute_error: 2.7967 - val_loss: 9.1965 - val_mean_squared_error: 9.1965 - val_rmse: 3.0326 - val_mean_absolute_error: 2.5236 - lr: 7.8125e-06
Epoch 307/1000
3/3 [==============================] - 0s 18ms/step - loss: 12.5945 - mean_squared_error: 12.5945 - rmse: 3.5489 - mean_absolute_error: 2.7964 - val_loss: 9.1941 - val_mean_squared_error: 9.1941 - val_rmse: 3.0322 - val_mean_absolute_error: 2.5233 - lr: 7.8125e-06
Epoch 308/1000
1/3 [=========>....................] - ETA: 0s - loss: 13.9395 - mean_squared_error: 13.9395 - rmse: 3.7336 - mean_absolute_error: 2.9343
Epoch 00308: ReduceLRO

Epoch 333/1000
3/3 [==============================] - 0s 19ms/step - loss: 15.0948 - mean_squared_error: 15.0948 - rmse: 3.8852 - mean_absolute_error: 2.9878 - val_loss: 9.1763 - val_mean_squared_error: 9.1763 - val_rmse: 3.0292 - val_mean_absolute_error: 2.5207 - lr: 1.9531e-06
Epoch 334/1000
3/3 [==============================] - 0s 19ms/step - loss: 13.3261 - mean_squared_error: 13.3261 - rmse: 3.6505 - mean_absolute_error: 2.9073 - val_loss: 9.1751 - val_mean_squared_error: 9.1751 - val_rmse: 3.0290 - val_mean_absolute_error: 2.5206 - lr: 1.9531e-06
Epoch 335/1000
3/3 [==============================] - 0s 17ms/step - loss: 14.0323 - mean_squared_error: 14.0323 - rmse: 3.7460 - mean_absolute_error: 2.9040 - val_loss: 9.1741 - val_mean_squared_error: 9.1741 - val_rmse: 3.0289 - val_mean_absolute_error: 2.5204 - lr: 1.9531e-06
Epoch 336/1000
3/3 [==============================] - 0s 23ms/step - loss: 13.9749 - mean_squared_error: 13.9749 - rmse: 3.7383 - mean_absolute_error: 2.9390 - 

3/3 [==============================] - 0s 19ms/step - loss: 13.2981 - mean_squared_error: 13.2981 - rmse: 3.6467 - mean_absolute_error: 2.9257 - val_loss: 9.1679 - val_mean_squared_error: 9.1679 - val_rmse: 3.0279 - val_mean_absolute_error: 2.5195 - lr: 9.7656e-07
Epoch 362/1000
3/3 [==============================] - 0s 16ms/step - loss: 12.5250 - mean_squared_error: 12.5250 - rmse: 3.5391 - mean_absolute_error: 2.7849 - val_loss: 9.1681 - val_mean_squared_error: 9.1681 - val_rmse: 3.0279 - val_mean_absolute_error: 2.5196 - lr: 9.7656e-07
Epoch 363/1000
3/3 [==============================] - 0s 19ms/step - loss: 13.5466 - mean_squared_error: 13.5466 - rmse: 3.6806 - mean_absolute_error: 2.8867 - val_loss: 9.1681 - val_mean_squared_error: 9.1681 - val_rmse: 3.0279 - val_mean_absolute_error: 2.5196 - lr: 9.7656e-07
Epoch 364/1000
3/3 [==============================] - 0s 18ms/step - loss: 13.6973 - mean_squared_error: 13.6973 - rmse: 3.7010 - mean_absolute_error: 2.8998 - val_loss: 9.168

In [11]:
# We want to try different training sample sizes: s
# (1453, 1000, 700, 350, 175)